<a href="https://colab.research.google.com/github/syahidmid/seo-weekly-report/blob/main/SEO_Weekly_Report_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes

To Do List 
- Create new column for 1. Preparing for Media O step, let name it "Tag"

https://docs.google.com/spreadsheets/d/1SbEAGAKkdujQdI6NBGXNKlNoxqeledQe55432RoO7cI/edit#gid=1665442719

# 1. Preparing for Media O 

In [ ]:
import os
import shutil
import io 
import pandas as pd
from google.colab import files

In [ ]:
uploaded = files.upload()

In [ ]:
folder_path = '/content/new report'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

file_name = list(uploaded.keys())[0]
shutil.move(file_name, os.path.join(folder_path, file_name))


In [ ]:
def move_file(file_name, path):
    shutil.move(file_name, os.path.join(path, file_name))


In [ ]:
media_o = pd.read_csv(os.path.join(folder_path, file_name)) 

In [ ]:
if 'URL' not in media_o.columns or 'Keyword' not in media_o.columns:
    print("Error: Columns 'URL' and/or 'Keyword' not found in the dataframe.")
else:
    media_o['search_key'] = media_o['URL'] + ' | ' + media_o['Keyword']
    media_o['Current Position'] = ''
    media_o['Keyword found in SERPRobot'] = ''
    media_o.to_csv(os.path.join(folder_path, 'Media O.csv'), index=False)
    print("File 'Media O.csv' has been updated with new columns and saved to the 'new report' directory.")


# 2 Upload and clean SERP Robot Data

In [ ]:
uploaded = files.upload()

In [ ]:
# Gets a list of all uploaded file names, then iteratively moves each file into a specified ('/content/new report') using the shutil.move function.
file_names = list(uploaded.keys())
folder_path = '/content/new report'
for file_name in file_names:
    shutil.move(file_name, os.path.join(folder_path, file_name))

In [ ]:
all_data = pd.concat([pd.read_csv(os.path.join(folder_path, name)).assign(Tag=name) for name in file_names])

In [ ]:
all_data['Found SERP'] = all_data['Found SERP'].str.replace('https://', '')  # Cleaning 1
all_data['Found SERP'] = all_data['Found SERP'].str.split('#').str[0]  # Cleaning 2
all_data['search_key'] = all_data['Found SERP'] + ' | ' + all_data['Keyword for: (lifepal.co.id/media)']

In [ ]:
all_data.to_csv(os.path.join(folder_path, 'SERP All Data.csv'), index=False)
print("All file succesfully combined, cleaned, and new column 'search_key' added. The updated data is saved to the 'new report' directory as 'SERP All Data.csv'")

# 3 Check for Keyword Position

In [ ]:
all_data.rename(columns={'www.google.co.id current position': 'position'}, inplace=True)

In [ ]:
if 'Current Position' in media_o.columns:
    media_o.drop('Current Position', axis=1, inplace=True)


In [ ]:
media_o = media_o.merge(all_data[['search_key', 'position']], on='search_key', how='left').copy()


In [ ]:
media_o.rename(columns={'position': 'Current Position'}, inplace=True)


In [ ]:
media_o['Current Position'].fillna(100, inplace=True)
print("Successfully updated Current Position in Media O")


Successfully updated Current Position in Media O


In [ ]:
media_o['Keyword found in SERPRobot'] = media_o['Keyword'].isin(all_data['Keyword for: (lifepal.co.id/media)']).map({True: 'Yes', False: 'No'})


In [ ]:
media_o.to_csv('/content/new report/Media O.csv', index=False)
print("The position data has been added to the Media 0.CSV file!'")


The position data has been added to the Media 0.CSV file!'


# 4. Cleaning GA Data 

In [ ]:
# Minta pengguna untuk mengunggah file
uploaded = files.upload()

Saving Lead Health 2.csv to Lead Health 2.csv
Saving Lead Health 1.csv to Lead Health 1.csv


In [ ]:
# Membuat DataFrame kosong untuk menampung semua data
all_data = pd.DataFrame()
first_filename = None  # Inisialisasi variabel di luar loop


In [ ]:
# Iterasi setiap file yang diunggah
for i, filename in enumerate(uploaded.keys()):

    # Menyimpan file yang diupload
    with open(filename, 'wb') as f:
        f.write(uploaded[filename])


In [ ]:
    # Buat direktori baru jika belum ada
    if not os.path.exists('new report'):
        os.makedirs('new report')

    # Pindahkan file yang diupload ke direktori baru
    shutil.move(filename, 'new report/' + filename)


'new report/Lead Health 1.csv'

In [ ]:
    # Menginisiasi variabel
    header_row = 0

    # Baca file dan cari 'Landing Page'
    with open('new report/' + filename, 'r') as f:
        for row_no, line in enumerate(f):
            if 'Landing Page' in line:
                header_row = row_no
                break


In [ ]:
    # Gunakan header_row sebagai parameter dalam fungsi pd.read_csv
    data = pd.read_csv('new report/' + filename, skiprows=header_row)


In [ ]:
    # Hanya mempertahankan baris yang dimulai dengan '/media/'
    data = data[data.iloc[:,0].str.startswith('/media/', na=False)]


In [ ]:
    # Menghapus tanda tanya "?" dan semua karakter setelahnya di kolom 'Landing Page'
    data.iloc[:,0] = data.iloc[:,0].str.split('?').str[0]


In [ ]:
    # Menambah "lifepal.co.id" sebelum "/media/"
    data.iloc[:,0] = 'lifepal.co.id' + data.iloc[:,0]


In [ ]:
    # Menambahkan data dari file ini ke DataFrame keseluruhan
    all_data = pd.concat([all_data, data], ignore_index=True)

    # Jika ini adalah iterasi pertama, simpan nama file
    if i == 0:
        first_filename = os.path.splitext(filename)[0]


In [ ]:
# Menyimpan DataFrame ke file CSV baru dengan nama file pertama ditambah '_cleaned'
all_data.to_csv(f'new report/{first_filename}_cleaned.csv', index=False)
print(f"Data telah berhasil disimpan ke file 'new report/{first_filename}_cleaned.csv'")


# 5. Vlookup

In [ ]:
# Input the filename to search
filename = input("Enter the filename to search (e.g., SERP All Data.csv): ")


Enter the filename to search (e.g., SERP All Data.csv): /content/new report/None_cleaned.csv


In [ ]:
# Read the data file
all_data = pd.read_csv(filename)


In [ ]:
# Remove duplicates in the 'Landing Page' column
all_data.drop_duplicates(subset='Landing Page', inplace=True)


In [ ]:
# Print the first 5 rows of the data
print("Data in file '{}':".format(filename))
print(all_data.head(5))


In [ ]:
# Input the column name to extract
column_name = input("Enter the column name to extract: ")


Enter the column name to extract: Lead - Health (Goal 3 Completions)


In [ ]:
# Add the column to Media O
media_o[column_name] = ''


In [ ]:
# Perform vlookup to check if 'URL' in Media O appears in 'Landing Page' in all_data
if 'Landing Page' in all_data.columns and column_name in all_data.columns:
    media_o[column_name] = media_o['URL'].map(all_data.set_index('Landing Page')[column_name])


In [ ]:
    # Set 'Keyword found in SERPRobot' based on the vlookup result
    media_o['Keyword found in SERPRobot'] = media_o[column_name].notna().map({True: 'Yes', False: 'No'})


In [ ]:
    # Print success message
    print("Successfully performed vlookup and added the '{}' column to Media O.".format(column_name))
    # Save updated dataframe back to CSV
    media_o.to_csv('/content/new report/Media O.csv', index=False)


Successfully performed vlookup and added the 'Lead - Health (Goal 3 Completions)' column to Media O.


# Delete All Data

In [ ]:


def delete_files_in_folder(folder_path):
    # Melakukan pengecekan apakah folder tersebut ada
    if os.path.exists(folder_path):
        # Menghapus semua file dalam folder
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            # Pengecekan jika file tersebut adalah file dan bukan direktori lain
            if os.path.isfile(file_path):
                os.remove(file_path)  # Menghapus file
        print("All files in the folder have been deleted.")
    else:
        print("Folder does not exist.")

folder_path = "/content/new report"
delete_files_in_folder(folder_path)


All files in the folder have been deleted.
